# Efficient Yelp API Calls - Core Assignment

## Imports

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/klein/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
def create_json_file(JSON_FILE, delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If file DOES exist:
    if file_exists == True:

        # Check if user wants to delete is exists
        if delete_if_exists==True:

            print(f'[!] {JSON_FILE} already exists. Deleting previous file...')
            # Delete file and confirm it no longer exists
            os.remove(JSON_FILE)
            # recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f'[i] {JSON_FILE} already exists.')

    # If file does NOT exist:
    else:

        # Inform user and save empty list
        print(f'[i] {JSON_FILE} not found. Saving empty list to new file.')

        ## Create any needed folders
        # Get folder name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder
        if len(folder)>0:
            # Create the folder
            os.makedirs(folder, exist_ok=True)
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

In [5]:
# Specify JSON_FILE filename
# Include search terms in the filename
JSON_FILE = 'Data/results_in_progress_SA_pizza.json'

# Set API call parameters
LOCATION = 'San Antonio, TX'
TERM = 'Pizza'

In [6]:
# Create a new empty json file (explicitly delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
# Load previous results (should now be zero) and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

# Set offset based on previous results
n_results = len(previous_results) # Use this for the offset
print(f'- {n_results} previous results found.')

## Performing the API call
# Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=n_results)
# How many results in total?
total_results = results['total']
print(f'There were {total_results} total results found, including any duplicates.') 
# There should be zero results at this point, as we have a fresh file.

# How many results did we get details for in a single page?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total required number of pages of results
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
print(f'We would need {n_pages} pages to display all results.')

[!] Data/results_in_progress_SA_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_SA_pizza.json not found. Saving empty list to new file.
- 0 previous results found.
There were 1300 total results found, including any duplicates.
We would need 65 pages to display all results.


In [ ]:
for i in tqdm_notebook(range(1, n_pages+1)):

    ## PROGRESS OF QUERY SO FAR
    # Read in results in progess file and check length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    # Save number of results to use as offset
    n_results = len(previous_results)
    # Stop query if it would cause an error
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    ## ADDING TO FILE WITH NEXT PAGE OF QUERY RESULTS
    # Use n_results as the offset
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)
    # Append new results and save to file
    previous_results.extend(results['businesses'])
    # Display previous_results
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)

    time.sleep(0.1)

  0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
# Load in final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(2), final_df.tail(2))

In [ ]:
# Check for and remove any duplicates
final_df.duplicated(subset='id').sum()

In [ ]:
# Save the final results to a compressed csv
final_df.to_csv('Data/final_results_SA_pizza.csv.gz', compression='gzip', index=False)